In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
from kobert_tokenizer import KoBERTTokenizer
import random
from torch.utils.tensorboard import SummaryWriter

/home/kaleb/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kaleb/miniconda3/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
df = pd.read_csv('/home/kaleb/Qualcom_comp/data/hackathon_train.csv', encoding='cp949', index_col=0)

# split train and test dataframe
train_df_list = []
test_df_list = []
for idx in df['User_ID'].unique():
    train_df_list.append(df[df['User_ID']==idx][0:40])
    test_df_list.append(df[df['User_ID']==idx][40:])
    
train_df = pd.concat(train_df_list, ignore_index=True)
test_df = pd.concat(test_df_list, ignore_index=True)

In [3]:
# model_name = 'kykim/bert-kor-base'
# model_name = 'monologg/kobigbird-bert-base'
# model_name = 'beomi/kcbert-base'
# model_name = 'skt/kobert-base-v1'
model_name = 'snunlp/KR-BERT-char16424' # MAIN EMBEDDING BERT MODEL
# model_name = 'klue/bert-base'
# model_name = 'klue/roberta-base'

In [4]:
# load embedding
# train_result = torch.load(f'{model_name.replace("/", "_")}_train_snu_only_rep.pt')
# test_result = torch.load(f'{model_name.replace("/", "_")}_test_snu_only_rep.pt')
train_result = torch.load(f'/home/kaleb/Qualcom_comp/{model_name.replace("/", "_")}_train_embed_regular.pt')[0]
test_result = torch.load(f'/home/kaleb/Qualcom_comp/{model_name.replace("/", "_")}_test_embed_regular.pt')[0]

In [5]:
def set_random(SEED=0):
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

class MyDataset(Dataset):
    def __init__(self, data, label, label_idx=0):
        self.data = data
        self.label = label
        self.label_idx = label_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], torch.tensor(self.label[idx][self.label_idx])
    
def convert_mbti_to_label(mbti: str):
    """
    :param mbti: string. length=4
    :return:
    """
    stand = 'ISTJ'  # [0, 0, 0, 0]
    result = []
    for i in range(4):
        if stand[i] == mbti[i]:
            result.append(0)
        else:
            result.append(1)

    return result

# def convert_label_to_mbti(num, label_idx):
#     stand = 'ISTJ'
#     mbti = stand[label_idx]
    

In [6]:
def train(model, dl, optimizer, criterion, device=1):
    model = model.cuda(device)
    model.train()
    loss_all, acc_all = 0, 0
    
    for x, y in dl:
        x, y = x.cuda(device), y.cuda(device)
        output = model(x)
        loss = criterion(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(axis=1) == y).sum() / len(y)

        loss_all += loss.item()
        acc_all += acc.item()


    loss = loss_all / len(dl)
    acc = acc_all / len(dl)

    return loss, acc

def valid(model, dl, optimizer=None, criterion=None, device=1):
    model = model.cuda(device)
    model.eval()
    loss_all, acc_all = 0, 0
    
    output_list = []
    for x, y in dl:
        x, y = x.cuda(device), y.cuda(device)
        output = model(x)
        loss = criterion(output, y)

        acc = (output.argmax(axis=1) == y).sum() / len(y)

        loss_all += loss.item()
        acc_all += acc.item()

        output_list.append(output.argmax(dim=1).cpu())
        
    loss = loss_all / len(dl)
    acc = acc_all / len(dl)
    
    
#     # userid accuracy
#     result = 0
#     a = torch.cat(output_list)
#     for uid in test_df['User_ID'].unique():
#         idx = test_df[test_df['User_ID']==uid].index
#         if a[idx].count_nonzero().item() > len(a[idx])//2:
#             label = 1
#         else:
#             label = 0
            
#         result += convert_mbti_to_label(test_df[test_df['User_ID']==uid]['MBTI'].unique()[0])[label_idx] == label
        
    
    return loss, acc


In [7]:
def forward(model, dl, device=0):
    pooled = []
    hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=True)
        p, h = output.pooler_output, output.hidden_states
        pooled.append(p) # pooler output
        hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
    return torch.cat(pooled), torch.cat(hidden)

In [8]:
train_df['One_word_answer'] = train_df['Answer'].apply(lambda x: (x.split('>')[0])[x.index('<')+1:])
test_df['One_word_answer'] = test_df['Answer'].apply(lambda x: (x.split('>')[0])[x.index('<')+1:])
# train_df['One_word_answer'] = train_df['One_word_answer'].astype('category').cat.codes
# train_df['One_word_answer'].unique()

train_df['One_word_answer'] = train_df['One_word_answer'].apply(lambda x : 1 if x.startswith('중') else (2 if x.startswith('아') or x.startswith('어') else 0))
test_df['One_word_answer'] = test_df['One_word_answer'].apply(lambda x : 1 if x.startswith('중') else (2 if x.startswith('아') or x.startswith('어') else 0))

train_df['Answer_rep'] = train_df['Answer'].apply(lambda x: x.replace(f"<{(x.split('>')[0])[x.index('<')+1:]}>", ''))
test_df['Answer_rep'] = test_df['Answer'].apply(lambda x: x.replace(f"<{(x.split('>')[0])[x.index('<')+1:]}>", ''))

# train_df = pd.get_dummies(train_df, columns=['One_word_answer'], prefix='is')
# test_df = pd.get_dummies(test_df, columns=['One_word_answer'], prefix='is')

# train_df = pd.get_dummies(train_df, columns=['Age'], prefix='is')
# test_df = pd.get_dummies(test_df, columns=['Age'], prefix='is')
# train_df['One_word_ans_enc'].unique()

In [9]:
# for col in train_df.columns:
#     if col not in test_df.columns:
#         test_df[col] = False

In [10]:
# added_cols = ['Gender', 'Age'] + [col for col in train_df.columns if col.startswith('is')]

added_cols = []
nums = 48
for _ in range(nums):
    added_cols += ['Gender']
for _ in range(nums):
    added_cols += ['Age']
for _ in range(nums//3):
    added_cols += [col for col in train_df.columns if col.startswith('is')]


col_data = train_df[added_cols].values
test_col_data = test_df[added_cols].values

# print(col_data[0][0].shape)
col_data = col_data.astype(float)
test_col_data = test_col_data.astype(float)


col_data = torch.tensor(col_data, dtype=torch.float, device='cuda:0')
test_col_data = torch.tensor(test_col_data, dtype=torch.float, device='cuda:1')


train_result_added = torch.cat([train_result, col_data], dim=1)
test_result_added = torch.cat([test_result, test_col_data], dim=1)


In [11]:
train_result_added.shape, test_result_added.shape

(torch.Size([9600, 864]), torch.Size([1920, 864]))

In [12]:
train_df

,User_ID,Gender,Age,MBTI,Q_number,Answer,One_word_answer,Answer_rep
0,1,1,30,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...,2,어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의 친구와만...
1,1,1,30,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...,1,다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며 요즘...
2,1,1,30,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...,0,감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 영화관도 ...
3,1,1,30,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...,1,대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고 변수가...
4,1,1,30,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...,2,평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대로 못합니...
...,...,...,...,...,...,...,...,...
9595,240,0,40,ISTJ,36,<아니다> 저는 즐거운 파티나 행사로 일주일 피로를 푸는 편이 아닙니다. 이유는 그...,2,저는 즐거운 파티나 행사로 일주일 피로를 푸는 편이 아닙니다. 이유는 그런 식의 ...
9596,240,0,40,ISTJ,37,<중립> 저는 미술관 가는 일을 좋아하지 않습니다. 이유는 미술 작품에 관심이 많이...,1,저는 미술관 가는 일을 좋아하지 않습니다. 이유는 미술 작품에 관심이 많이 없기 ...
9597,240,0,40,ISTJ,38,<그렇다> 저는 다른 사람의 감정을 이해하기 힘들 때가 많습니다. 이유는 각자의 살...,0,저는 다른 사람의 감정을 이해하기 힘들 때가 많습니다. 이유는 각자의 살아온 환경...
9598,240,0,40,ISTJ,39,<아니다> 저는 매일 할 일을 계획하지 않습니다. 이유는 매일매일보다는 크게 한 건...,2,저는 매일 할 일을 계획하지 않습니다. 이유는 매일매일보다는 크게 한 건 방식으로...


In [13]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from thundersvm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
import joblib
# import AUC
from sklearn.metrics import roc_auc_score
from cuml import *
import cudf

gpu_id = 0
cudf.set_allocator("managed")
# cudf.cuda.select_device(gpu_id)

train_data = train_result_added
test_data = test_result_added

# data_merged = torch.cat([train_data, test_data], dim=0)


def main(label_idx, train_data_func, test_data_func, model):
    # dataset / dataloader
    # quantile_transformer = preprocessing.QuantileTransformer(n_quantiles = 100)#, output_distribution = 'uniform')
    
    train_data = train_data_func.detach().cpu().numpy()
    # train_data = quantile_transformer.fit_transform(train_data)
    # joblib.dump(quantile_transformer, f'./quantile_transformer_{label_idx}.pkl')
    
    
    train_label = train_df['MBTI'].map(convert_mbti_to_label)
    train_label = np.array([train_lab[label_idx] for train_lab in train_label])
    
    test_data = test_data_func.detach().cpu().numpy()
    # test_data = quantile_transformer.transform(test_data)
    test_label = test_df['MBTI'].map(convert_mbti_to_label)
    test_label = np.array([test_lab[label_idx] for test_lab in test_label])
    
    
    data_merged = np.concatenate([train_data, test_data], axis=0)
    label_merged = np.concatenate([train_label, test_label], axis=0)    
    
    
    model.fit(train_data, train_label)
    # model.fit(data_merged, label_merged) # we just uncomment this line to train on all data
    # pred_train = model.predict(train_data)
    
    # MAIN
    # prediction = model.predict(test_data)
    # best_accuracy = accuracy_score(test_label, prediction)
    
    
    prediction = model.predict_proba(test_data)
    best_roc = roc_auc_score(test_label, prediction[:, 1])
    
    
    
    return best_roc, model#, pred_train, prediction, train_label, test_label

/home/kaleb/miniconda3/lib/python3.10/site-packages/cudf/utils/utils.py:218: FutureWarning: The cudf.set_allocator function is deprecated and will be removed in a future release. Please use rmm.reinitialize (https://docs.rapids.ai/api/rmm/stable/api.html#rmm.reinitialize) instead. Note that `cudf.set_allocator(allocator="managed")` is equivalent to `rmm.reinitialize(managed_memory=True)`.
  warnings.warn(


In [14]:
import csv
import pickle
MBTI = ['IE', 'SN', 'TF', 'JP']
set_random(42)
best_accuracies = {}
models = list()

# pred_train_n = []
# pred_test_n = []


model = SVC(kernel='rbf', C=1, gamma=0.085, probability=True)
    
# model = DecisionTreeClassifier(max_depth=10)
# model = RandomForestClassifier(max_depth=10, n_estimators=100, random_state=0)
# model = XGBClassifier(max_depth=10, n_estimators=100, random_state=0)
# model = LogisticRegression(C = 0.1)
# model = GradientBoostingClassifier(max_depth=10, n_estimators=100, random_state=0)
# model = KNeighborsClassifier(n_neighbors=5)
for i in range(4):
    
    result, model = main(i, train_data, test_data, model)
    best_accuracies[MBTI[i]] = result
    models.append(model)
    # print(f'Best accuracy for {C} and {gamma} {MBTI[i]}: {result}')
        
    # result, model = main(i, train_data, test_data)
    # best_accuracies[MBTI[i]] = result
    # models.append(model)
    print(f'Best accuracy for {MBTI[i]}: {result}')



avg = np.mean(list(best_accuracies.values()))
print('avg is ', avg)

Best accuracy for IE: 0.7586566840277778
Best accuracy for SN: 0.6845019531250001
Best accuracy for TF: 0.6908170572916666
Best accuracy for JP: 0.7098187934027778
avg is  0.7109486219618055


In [15]:
print(f'Best accuracy for each MBTI:\n {best_accuracies}')
# final average accuracy of the 4 models
final_average_accuracy = sum(best_accuracies.values()) / len(best_accuracies)
print(f'Final average accuracy on validation dataset: {final_average_accuracy}')

Best accuracy for each MBTI:
 {'IE': 0.7586566840277778, 'SN': 0.6845019531250001, 'TF': 0.6908170572916666, 'JP': 0.7098187934027778}
Final average accuracy on validation dataset: 0.7109486219618055
